In [1]:
import pandas as pd
import datetime
import requests
import warnings
import socket
import json

from urllib.request import urlopen
from urllib.error import URLError

warnings.filterwarnings("ignore")

In [2]:
server = 'http://10.11.160.83/'
nodelocalhost = 'http://localhost:3000/'
post_entry_url = 'http://localhost:3000/addentry'

headers = { 'Content-Type' : 'application/json'}

In [3]:
def server_reachable():
    try:
        urlopen(nodelocalhost, timeout=5)
        return True
    except URLError as err:
        return False
    except socket.timeout:
        print ("Server timeout")
        return False

In [4]:
server_reachable()

True

In [5]:
def change_data_type (dataframe, columns, data_type):
    for column in columns:
        dataframe [column] = dataframe[column].astype(data_type)

def calculate_percentage (dataframe, column_1, column_2):
    dataframe.apply (lambda row: (row.column_1 / row.column_2 )*100, axis=1)

In [6]:
server_reachable()

True

In [89]:
data = pd.read_csv('extractData.txt', sep="|", header=None)
data = data.fillna(0)
data = data [:-1]
data.columns = ["manifest_name", "begin_date", "end_date", "servedHH", "servedPopn", "plannedHH", "plannedPopn"]

data['percent_HH'] = data.apply(lambda row: (row.servedHH / row.plannedHH)*100, axis=1)
data['percent_Popn'] = data.apply(lambda row: (row.servedPopn / row.plannedPopn)*100, axis=1)

columns = ['percent_HH', 'percent_Popn', 'servedHH' , 'servedPopn', 'plannedHH' , 'plannedPopn' ]

change_data_type (data, columns, 'int64')
change_data_type (data, columns, 'str')

data['hostname'] = socket.gethostname()
data[['settlement','cycle', 'cycle_year', 'mod', 'fdp', 'modality']] = data.manifest_name.apply(lambda x: pd.Series(str(x).split("-", 5)))
data = data.drop(['mod'], axis =1)

In [90]:
data.head()

,manifest_name,begin_date,end_date,servedHH,servedPopn,plannedHH,plannedPopn,percent_HH,percent_Popn,hostname,settlement,cycle,cycle_year,fdp,modality
0,Rhino-07-19-c-Tika-Cash,2019-07-11,2019-08-09,0,0,667,2916,0,0,ubuntu,Rhino,07,19,Tika,Cash
1,Rhino-07-19-c-Omugo FDP2-Cash,2019-07-11,2019-08-16,0,0,1612,4513,0,0,ubuntu,Rhino,07,19,Omugo FDP2,Cash
2,Rhino-07-19-f-Odubu-Food,2019-07-11,2019-08-09,0,0,638,2814,0,0,ubuntu,Rhino,07,19,Odubu,Food
3,Rhino-07-19-c-Odubu-Cash,2019-07-11,2019-07-11,0,0,1204,7429,0,0,ubuntu,Rhino,07,19,Odubu,Cash
4,Rhino-07-19-f-Tika-Food,2019-07-11,2019-08-16,0,0,2007,6617,0,0,ubuntu,Rhino,07,19,Tika,Food


In [91]:

area = []
field = []
    
for row in data['settlement']:
    if row == 'Lobule' or 'Rhino' or 'Imvepi' or 'Bidibidi':
        area.append('Arua')
        field.append ('Koboko')
    elif row == 'Adjumani' or 'Palabek' or 'Palorinya' or 'Kiryandongo':
        area.append('Arua')
        field.append ('Gulu')
    elif row == 'Nakivale' or 'Oruchinga':
        area.append('Mbarara')
        field.append ('Isingiro')
    elif row == 'Rwamwanja' or 'Kyaka':
        area.append('Mbarara')
        field.append ('Kyenjojo')
    elif row == 'Kyangwali':
        area.append('Mbarara')
        field.append ('Kyangwali')
    else:
        area.append('None')
        field.append ('None')
       
# Create a column from the list
data['area'] = area
data['field'] = field 

data.to_json('extractData.json', orient='records')
print ("Wrangles")

Wrangles


In [85]:
json_string = data.to_json(orient='records')

In [86]:
json_data = json.loads(json_string)

In [88]:
for entry in json_data:
    r = requests.post(post_entry_url, data=json.dumps(entry), headers=headers)
    print (r)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [84]:
gs = json.loads(thing)
for entry in gs:
    print (entry)

{'manifest_name': 'Rhino-07-19-c-Tika-Cash', 'begin_date': '2019-07-11', 'end_date': '2019-08-09', 'servedHH': '0', 'servedPopn': '0', 'plannedHH': '667', 'plannedPopn': '2916', 'percent_HH': '0', 'percent_Popn': '0', 'hostname': 'ubuntu', 'settlement': 'Rhino', 'cycle': '07', 'cycle_year': '19', 'fdp': 'Tika', 'modality': 'Cash', 'area': 'Arua', 'field': 'Koboko'}
{'manifest_name': 'Rhino-07-19-c-Omugo FDP2-Cash', 'begin_date': '2019-07-11', 'end_date': '2019-08-16', 'servedHH': '0', 'servedPopn': '0', 'plannedHH': '1612', 'plannedPopn': '4513', 'percent_HH': '0', 'percent_Popn': '0', 'hostname': 'ubuntu', 'settlement': 'Rhino', 'cycle': '07', 'cycle_year': '19', 'fdp': 'Omugo FDP2', 'modality': 'Cash', 'area': 'Arua', 'field': 'Koboko'}
{'manifest_name': 'Rhino-07-19-f-Odubu-Food', 'begin_date': '2019-07-11', 'end_date': '2019-08-09', 'servedHH': '0', 'servedPopn': '0', 'plannedHH': '638', 'plannedPopn': '2814', 'percent_HH': '0', 'percent_Popn': '0', 'hostname': 'ubuntu', 'settlemen

In [67]:
requests.post(post_entry_url, json=thing, headers=headers)

<Response [400]>

In [73]:
thing2 = thing[1:]
thing2 = thing2[:-1]

In [79]:
response = requests.post(post_entry_url, data=json.dumps(thing), headers=headers)
print (response)

<Response [400]>


In [64]:
datum = {"manifest_name" : "py_test"}
for x in range (1,10): 
    requests.post(post_entry_url, json=datum, headers=headers)

In [55]:
with open('extractData.json', 'rb') as json_file:
    json_data = json.load(json_file)

In [56]:
r = requests.post(post_entry_url, data=json.dumps(json_data), headers=headers)
print (r)

<Response [200]>


In [59]:
contents = open('extractData.json', 'rb').read()

In [61]:
print (json.dumps(contents))

TypeError: Object of type 'bytes' is not JSON serializable

<_io.BufferedReader name='extractData.json'>


In [57]:
# with open('extractData.json') as json_file:
#     data_json = json.load(json_file)
#     for entry in data_json:
#         print (entry)

In [58]:
# with open('extractData.json') as json_file:
#     data_json = json.load(json_file)
#     for entry in data_json:
# #         print (entry)
#         requests.post(post_entry_url, json=entry, headers=headers)

with open('extractData.json') as json_file:
    data_json = json.load(json_file)
    for entry in data_json:
        r = requests.post(post_entry_url, data = json.dumps(entry), headers=headers)
    print ("Completed posting data to server at " + str(datetime.datetime.now))